In [1]:
import sys
import collections
from slither.slither import Slither
from slither.slithir.operations.event_call import EventCall
from slither.slithir.operations.index import Index as IndexOperation
from slither.slithir.operations.binary import Binary as BinaryOperation
from slither.slithir.operations.solidity_call import SolidityCall as SolidityCallOperation
from slither.solc_parsing.variables.state_variable import StateVariableSolc
from slither.core.solidity_types.mapping_type import MappingType
from constants import (
    ERC721_EVENT_SIGNATURES,
    ERC721_FX_SIGNATURES,
    ERC721_ON_RECEIVE_SIGNATURES,
    ERC721_GETTERS,
    ERC721_EVENT_BY_FX,
    ALLOWANCE_FRONTRUN_FX_SIGNATURES,
    ALLOWANCE_FRONTRUN_EVENT_BY_FX,
)
from log import (
    log_matches,
    log_event_per_function,
    log_modifiers_per_function,
    log_approve_checking_balance
)

In [2]:
filename = 'test\contracts\\token\ERC721\ERC721.sol'
contract_name = 'ERC721'
slither = Slither(filename)

# Get an instance of the contract to be analyzed
contract = slither.get_contract_from_name(contract_name)
if not contract:
    print(f"Contract {contract_name} not found")
    print("Either you mispelled the contract's name or solc cannot compile the contract.")
    exit(-1)

In [13]:
def get_contract_from_name(slither, contract_name):
    contract = slither.get_contract_from_name(contract_name)
    return contract if type(contract) is not list else contract[0]

In [15]:
approve_func = [func for func in contract.functions if func.name == 'approve']

In [39]:
print(approve_func[1].nodes[5])

EXPRESSION Approval(owner,to,tokenId)


In [40]:
def is_event_call(obj):
    return isinstance(obj, EventCall)

def get_events(function):
    for node in getattr(function, 'nodes', []):
        for ir in node.irs:
            if is_event_call(ir):
                yield ir

In [41]:
approve_events = list(get_events(approve_func[1]))
print(approve_events[0].name)

Approval


In [53]:
def is_visible(function):
    return is_public(function) or is_external(function)

def is_external(function):
    return function.visibility == "external"


def is_public(element):
    return element.visibility == "public"

def get_visible_functions(functions):
    return [f for f in functions if is_visible(f)]

def is_interface(contract):
    return contract.contract_kind == "interface"
def get_implemented_functions(functions):
    return [f for f in functions if not is_interface(f.contract_declarer) and f.nodes]

In [54]:
all_functions = get_implemented_functions(contract.functions)
visible_functions = get_visible_functions(
    get_implemented_functions(contract.functions)
)

In [55]:
for f in visible_functions:
    print(f.name)

supportsInterface
constructor
balanceOf
ownerOf
approve
getApproved
setApprovalForAll
isApprovedForAll
transferFrom
safeTransferFrom
safeTransferFrom
